In [ ]:
# want to see the images inline
# %matplotlib inline

In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from scipy.stats import gamma
from scipy.optimize import curve_fit
from scipy import special

from configparser import ConfigParser, ExtendedInterpolation
import utils as utils

from tqdm import tqdm_notebook as tqdm

In [ ]:
# read config file 
config = ConfigParser(interpolation=ExtendedInterpolation())
config.read('config.ini')

In [ ]:
PARAMS = config.get('main', 'PARAMS')

In [ ]:
PARAMS

In [ ]:
FOLDER = config.get(PARAMS, 'FOLDER')
EXPERIMENT = config.get(PARAMS, 'EXPERIMENT')
DB_FILENAME = config.get(PARAMS, 'DB_FILENAME')
TYPE = config.get(PARAMS, 'TYPE')
COLOR = config.get(PARAMS, 'COLOR')

In [ ]:
FOLDER

In [ ]:
# use load_i_z(file_path, skiprows_=1)
# returns I, z

In [ ]:
# returns the y coordinate for the given line
# y = kx + b; also works with vectors
def f_x(x_, k_, b_):
    return k_*x_ + b_

In [ ]:
def g_x(a, x):
    return polyval(a, x)

In [ ]:
# important indices
type_columns = ['c0_type', 'c1_type', 'c2_type']
stain_prefix = np.array([['C0-', 'C1-', 'C2-', 'C3-', 'C4-']])
filename_column = 'cropped_image_file'

In [ ]:
# read the db and parse images that we want to process
df_path = os.path.join(FOLDER, 'smFISH-database', DB_FILENAME)
df = pd.read_csv(df_path, 
                 sep=',', 
                )

In [ ]:
# TODO: maybe it is necessary to fill in other values here, too
# fix missing entries in the colmns that we are planning to use 
df['cropped_image_file'].fillna('', inplace=True)
df['c0_type'].fillna('', inplace=True)
df['c1_type'].fillna('', inplace=True)
df['c2_type'].fillna('', inplace=True)

In [ ]:
dff = df[df['cropped_image_file'].apply(lambda x: x.startswith(EXPERIMENT))]

In [ ]:
# seems to be working
row, col = np.where(dff[type_columns].applymap(lambda x: x == TYPE))
n_samples = dff.shape[0]
new_prefix = np.repeat(stain_prefix, n_samples, axis=0)[row, col]
new_filename = dff[filename_column].values[row]
full_filenames = ["{}{}".format(a_, b_[:-4]) for a_, b_ in zip(new_prefix, new_filename)]

In [ ]:
dataset = []
pbar = tqdm(total=len(full_filenames))
for ff in full_filenames: 
    tmp = os.path.join(FOLDER, 'csv-before', ff + '.csv')
    dataset.append(tmp)
    pbar.update(1)
pbar.close()

In [ ]:
print(f'Processing: {len(dataset)} files') 

pbar = tqdm(total=len(dataset))
for d in dataset: 
    pbar.update(1)
    if(not os.path.exists(d)):
        continue
        
    try:
        # create the canvas
        fig = plt.figure(figsize=(8,5))
        title = utils.create_title(d, name_id=-1)
        fig.suptitle(title + " / " + TYPE)
        
        # load the data and scale it accordingly
        I, z = utils.load_i_z(d, skiprows_=0)
        
        # load parameters
        p_path = d.replace('csv-before', 'csv-parameters')
        coeff = utils.load_params(p_path, skiprows_=0)
        
        plt.close()
    except(ValueError, StopIteration, RuntimeError):
        print("Caught the error for you")    
        
pbar.close()

In [ ]:
# actual plotting 
for idx in range(0, len(dataset)):
    # feedback
    
    if(not os.path.exists(dataset[idx])):
        # print("doesn't exist")
        continue
    
    try:
        # print("Processing:", dataset[idx]) 
        z,I = load_data(dataset[idx])

        # load the parameters from file
        coeff = load_params(p_dataset[idx])

        print(coeff)

        plt.figure(figsize=(8,5))

        title = create_title(dataset[idx], name_id=5)
        plt.title(title + " / " + labels[0])

        x_fit = np.linspace(np.min(z), np.max(z))
        y_fit = g_x(coeff[::-1], x_fit)

        plt.plot(z, I, 'o', color=color)
        plt.plot(x_fit, y_fit, linewidth=5, color='#66A5AD')

        x_min = np.min(x_fit)
        y_min = np.min(y_fit)

        x_limits = [0, 120]
        y_limits = [-0.2, 1.0]

        axes = plt.gca()
        axes.set_xlim(x_limits)
        axes.set_ylim(y_limits)

        plt.xlabel('z-slice')
        plt.ylabel('intensity')

        info_text = "Total: " + str(I.shape[0]) + "\n" + "Coeff: " + str(['%.2e' % elem for elem in coeff])
        plt.text(x_limits[0] + (x_limits[1] - x_limits[0])*0.02, y_limits[0] + (y_limits[1] - y_limits[0])*0.04, info_text, color='black', bbox=dict(facecolor='white', alpha=1))

        # plt.legend(loc = 'upper right')      
        full_path_to_use = folder_path + "img-before/" + labels[0] + "/" 
        if not os.path.exists(full_path_to_use):
            os.makedirs(full_path_to_use)
        plt.savefig(full_path_to_use + title + ".pdf")

        plt.show()
    except(ValueError, StopIteration, RuntimeError):
        print("Caught the error for you")

In [ ]:
# function to normalize the z-correction curve 
def norm_z(x, y):
    scale = np.max(x) - np.min(x)
    x_normed = (x - np.min(x))/scale
    y_normed = y / scale 
    return (x_normed, y_normed)

In [ ]:
# shift so that intensity value starts at val = 0.5
def shift_z(y, top = 0.5):
    y_shift = y - np.max(y) + top 
    return y_shift

In [ ]:
# have to figure aout what are the min-max value
z_min = sys.float_info.max
z_max = -sys.float_info.max

for idx in range(0, len(dataset)):
    if(not os.path.exists(dataset[idx])):
        # print("doesn't exist")
        continue
    try:
        z,I = load_data(dataset[idx])
        if (np.max(z) - np.min(z) > z_max):
            z_max = np.max(z) - np.min(z)
    except(ValueError, StopIteration, RuntimeError):
        print("Caught the error for you")
        

# plot of all ovelaping images
# actual plotting 
plt.figure(figsize=(8,5))
for idx in range(0, len(dataset)):
    # feedback
    
    if(not os.path.exists(dataset[idx])):
        # print("doesn't exist")
        continue
    
    try:
        # print("Processing:", dataset[idx]) 
        z,I = load_data(dataset[idx])

        # load the parameters from file
        coeff = load_params(p_dataset[idx])

        print(title, ":", coeff)  
        print('from {0} to {1}'.format(np.min(z), np.max(z)))
        title = create_title(dataset[idx], name_id=5)
        plt.title(labels[0])

        x_fit = np.linspace(np.min(z), np.max(z)) # np.linspace(0, 60) 
        fix_factor = g_x(coeff[::-1], np.min(z)) / g_x(coeff[::-1], x_fit)
        print (1/fix_factor)

        # always shift the min so that it is located at min
        x_fit_shifted = x_fit - np.min(z) 
        x_fit_shifted_normed = (x_fit_shifted - 0)/(z_max - 0)

        x_limits = [-0.05, 1.05]
        y_limits = [0.35, 1.05]

        axes = plt.gca()
        axes.set_xlim(x_limits)
        axes.set_ylim(y_limits)

        plt.xlabel('z-slice')
        plt.ylabel('intensity')

        # plt.plot(x_fit, y_fit, linewidth=5, label=title)
        plt.plot(x_fit_shifted_normed, 1/fix_factor, linewidth=5, label=title)
    
    except(ValueError, StopIteration, RuntimeError):
        print("Caught the error for you")
        
full_path_to_use = folder_path + "img-full/" + labels[0] + "/" 
if not os.path.exists(full_path_to_use):
    os.makedirs(full_path_to_use)
plt.savefig(full_path_to_use + (title[3:9] + "-" + labels[0]) + ".pdf")
    
# plt.legend(loc = 'upper right')
plt.show()